**Peliculas/Cine**

Topicos:
- Recursos
- Condiciónes de evnetos
- Eventos compartidos

**Enunciado**:

Modelar una sala de cine  con un mostrador para la venta de entradas a tres tipos de peliculas diferentes, tener en cuenta que se modela solo para un espectáculo.
- Las personas llegan en momentos aleatorios e intentan comprar un numero aleatorio de boletos (1-6) para una pelicula al azar.

- Cuando se agota las entradas a una pelicula, las personas que esperan por un boleto salen de la cola.

- La sala de cine es un Contenedor que abarca todos los procesos.
- El mostrador es un Recurso con la capacidad de uno.
- Finalmente, si la ultima persona desea comprar más boletos de los existentes se le vende solo los que tiene el mostrador.


In [1]:
import collections
import random
import simpy

VELOCIDAD_LLEGADA = 42
NUM_BOLETO = 50
TIEMPO_SIMULACION = 120
QUEUE={}

def ventaBoletos(env, num_boletos, pelicula, teatro, id):
  with teatro.contador.request() as turno:
    resultado = yield turno | teatro.sold_out[pelicula]
    if turno not in resultado:
      teatro.num_renegados[pelicula] += 1
      QUEUE[id]=[-1,env.now, '', 0]
      return      

    if teatro.num_boletos[pelicula] < num_boletos:
      yield env.timeout(0.5) 
      QUEUE[id]=[-1,env.now, '', 0]
      return
    else:
      QUEUE[id]=[1,env.now, pelicula, num_boletos]

    teatro.num_boletos[pelicula] -= num_boletos
    if teatro.num_boletos[pelicula] < 2:
      teatro.sold_out[pelicula].succeed()
      teatro.tiempo_agotado[pelicula] = env.now
      teatro.num_boletos[pelicula] = 0
    
    yield env.timeout(1)


def llegadaClientes(env, teatro):
  id=0
  while True:
    yield env.timeout(random.expovariate(1/0.5))
    pelicula = random.choices(teatro.peliculas, teatro.probabilidad, k=1)
    num_boletos = random.randint(1, 6)
    if teatro.num_boletos[pelicula[0]]:
      env.process(ventaBoletos(env, num_boletos, pelicula[0], teatro, id))
      id+=1


Teatro = collections.namedtuple('Teatro', 'contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados')

print('Teatro Carlos Crespi - UPS')
env = simpy.Environment()

contador = simpy.Resource(env,capacity=1)
peliculas = ['Conjuro 3', 'Rapidos y Furiosos 10', 'Pulp Fictions']
probabilidad=[0.1, 0.3, 0.6]
num_boletos = {pelicula: NUM_BOLETO for pelicula in peliculas}
sold_out = {pelicula: env.event() for pelicula in peliculas}
tiempo_agotado = {pelicula: None for pelicula in peliculas}
num_renegados = {pelicula: 0 for pelicula in peliculas}

teatro = Teatro(contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados)
env.process(llegadaClientes(env, teatro))
env.run(until=TIEMPO_SIMULACION)

# Analisis y resultados
for pelicula in peliculas:
  if teatro.sold_out[pelicula]:
    print('Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' %(pelicula, teatro.tiempo_agotado[pelicula]))
    print('Numero de personas que salieron de la fila/renegados %s' %teatro.num_renegados[pelicula])


Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 55.7 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 30.6 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 7
Pelicula: Pulp Fictions se agoto en el tiempo 25.6 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 13


**Tarea/Practica**

Al modelo de simulación de la venta de boletos de un teatro/cine agregar la simulación grafica 2D/3D utilizando cualquier libreria grafica, por ejemplo pygame/tkinter, etc. generar un informe en formato PDF que me permita evidenciar la simulación y generar una grafica/reporte de los resultados.

In [5]:
import simpy
import tkinter as tk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.gridspec as gridspec

class ClockAndData: 
    def __init__(self, canvas, peliculas, y1, y2, y3, time): 
        self.y1 = y1
        self.y2 = y2
        self.y3 = y3
        self.peliculas = peliculas
        self.canvas = canvas
        self.step_acum=0
        self.step_x=[0]
        self.step_y=[0]
        data_plot.draw()
        self.canvas.update() 

    def tick(self, time, client): 

        a1.cla()

        if client[0] == 1 and client[2] == self.peliculas[0]:
            self.y1 = self.y1+client[3]
        
        if client[0] == 1 and client[2] == self.peliculas[1]:
            self.y2 = self.y2+client[3]
        
        if client[0] == 1 and client[2] == self.peliculas[2]:
            self.y3 = self.y3+client[3]
        
        bars_y = [self.y1, self.y2, self.y3]

        a3.cla()
        a3.set_xlabel("Peliculas ofrecidas")
        a3.set_ylabel("Cantidad de boletos")
        a3.bar(peliculas, bars_y)

        a1.cla()

        self.step_acum+=client[0]
        self.step_x.append(time)
        self.step_y.append(self.step_acum)

        a1.set_xlabel("Tiempo de llegada")
        a1.set_ylabel("Progreso de la cola")
        a1.step(self.step_x, self.step_y)

        data_plot.draw()
        self.canvas.update() 
        # Re-draw the the clock and data fields on the canvas. Also update the Matplotlib charts. 

main = tk.Tk()
main.title("Simulacion Cine")
main.config(bg="#fff")

top_frame = tk.Frame(main)
top_frame.pack(side=tk.TOP, expand = False)

canvas = tk.Canvas(main, width = 1300, height = 150, bg = "white")
canvas.pack(side=tk.BOTTOM, expand = False)

f = plt.Figure()
gs = gridspec.GridSpec(1, 2,width_ratios=[1,1])
a3 = f.add_subplot(gs[0])
a3.plot()
a1 = f.add_subplot(gs[1])
a1.plot()

data_plot = FigureCanvasTkAgg(f, master=main)
data_plot.get_tk_widget().config(height = 400)
data_plot.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

canvas.pack()

clock = ClockAndData(canvas, peliculas, 0, 0, 0, 0) 

def create_clock(env):
    i=0
    t=QUEUE[i][1]

    while True: 
        yield env.timeout(t)
        
        try:
            clock.tick(env.now, QUEUE[i])
            i+=1
            t = QUEUE[i][1] - QUEUE[i-1][1]
            if t<0:
                t=0.1
        except:
            break            

env = simpy.Environment()
env.process(create_clock(env)) 
env.run(until = TIEMPO_SIMULACION)

main.mainloop()